In [ ]:
!pip install tabgan

     |████████████████████████████████| 44 kB 1.2 MB/s 
     |████████████████████████████████| 80 kB 3.9 MB/s 


In [ ]:
from tabgan.sampler import OriginalGenerator, GANGenerator
import pandas as pd
import numpy as np

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving landslide_nepal.csv to landslide_nepal.csv


In [ ]:
df = pd.read_csv('landslide_nepal.csv')
X = df.drop('output', axis=1)
y = df['output']

In [ ]:
df

,Unnamed: 0,YEAR,MONTH,DISTRICT,landslide_size,fatality_count,injury_count,latitude,longitude,PRECTOT,PS,QV2M,RH2M,T2M,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,output
0,0,2013,7,Syangja,medium,1.0,0.0,27.914200,83.787600,413.84,85.41,17.31,88.30,22.02,19.70,25.15,19.24,5.90,22.44,2.11,4.06,0.46,3.61,2.36,3.84,0.74,3.10,1
1,1,2017,6,Kaski,medium,0.0,1.0,28.281201,83.864415,220.45,77.91,12.83,74.56,18.58,13.26,22.78,14.70,8.08,19.22,2.25,4.49,0.41,4.08,2.21,3.79,0.55,3.25,1
2,2,2004,6,Salyan,no,0.0,0.0,28.400000,82.100000,214.13,88.50,13.91,59.08,26.01,16.80,31.00,21.43,9.57,26.71,3.05,5.41,1.06,4.36,3.95,5.92,1.67,4.25,0
3,3,2015,8,Kathmandu,medium,1.0,0.0,28.029700,85.093400,388.41,83.61,16.86,90.95,20.86,18.92,23.92,18.36,5.57,21.23,2.25,4.48,0.58,3.90,2.49,4.31,0.88,3.43,1
4,4,2017,3,Baglung,large,1.0,0.0,28.308297,83.174393,66.43,79.29,4.78,47.96,10.37,-0.91,17.07,4.87,12.19,9.36,2.37,4.95,0.47,4.48,2.41,4.48,0.52,3.96,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,510,2018,7,Bardiya,no,0.0,0.0,28.450000,81.300000,18.58,93.90,20.07,78.74,28.20,23.97,31.60,25.14,6.47,28.46,2.35,4.12,0.83,3.28,3.29,5.02,1.39,3.64,0
511,511,2013,7,Solukhumbu,medium,0.0,0.0,27.500100,86.866900,119.03,56.27,8.32,75.37,6.87,2.68,12.59,2.06,10.53,9.49,2.64,4.86,0.64,4.21,2.66,4.59,0.81,3.78,1
512,512,1990,4,Dhanusa,no,0.0,0.0,26.700000,85.900000,4.08,97.58,4.67,17.98,29.48,1.90,37.93,21.35,16.58,30.42,3.43,5.92,1.30,4.62,4.96,7.50,1.85,5.65,0
513,513,2007,8,Gulmi,medium,3.0,0.0,28.087700,83.097100,353.59,79.79,15.26,90.54,18.58,16.62,21.80,15.73,6.07,19.02,2.23,4.41,0.59,3.82,2.48,4.10,0.91,3.20,1


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
train.drop(['Unnamed: 0', 'YEAR', 'landslide_size', 'fatality_count', 'injury_count'], axis=1, inplace=True)
test.drop(['Unnamed: 0', 'YEAR', 'landslide_size', 'fatality_count', 'injury_count'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
target = train['output']
train.drop('output', axis=1, inplace = True)
test.drop('output', axis=1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
target = target.to_frame(name='output')

In [ ]:
train

,MONTH,DISTRICT,latitude,longitude,PRECTOT,PS,QV2M,RH2M,T2M,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
320,6,Kabhre,27.600000,85.600000,41.47,77.48,10.46,56.43,19.67,9.93,25.43,14.46,10.97,22.04,2.82,5.91,0.44,5.48,2.63,5.02,0.52,4.50
329,9,Kathmandu,27.416700,86.666700,54.10,83.84,14.95,86.72,19.74,16.92,23.44,16.86,6.58,19.92,1.73,3.73,0.38,3.35,1.83,3.28,0.58,2.70
173,10,Jumla,29.300000,82.200000,0.14,75.26,5.11,45.44,11.10,-0.98,18.81,5.50,13.30,10.26,2.24,4.84,0.35,4.50,2.24,4.26,0.42,3.84
272,7,Sindhuli,27.048300,86.324800,267.72,90.23,18.25,81.51,25.24,21.66,28.72,22.32,6.40,25.59,2.39,4.35,0.75,3.60,3.12,4.86,1.28,3.58
495,7,Bara,29.341370,81.309546,291.65,94.56,19.85,81.88,27.36,23.92,30.82,24.54,6.28,27.55,3.09,5.14,1.40,3.74,4.40,6.32,2.48,3.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,9,Rukum,28.777300,82.472800,169.43,74.69,12.32,87.05,14.72,12.13,18.78,11.43,7.36,14.98,1.92,4.03,0.38,3.65,1.95,3.50,0.48,3.02
106,9,Doti,29.300000,80.950000,30.30,83.42,12.02,63.72,21.05,13.16,27.07,16.32,10.74,21.27,2.08,4.44,0.36,4.08,2.26,4.22,0.46,3.76
270,9,Sankhuwasabha,27.366700,87.216700,169.43,81.82,14.18,87.74,18.27,15.64,21.71,15.68,6.04,18.57,2.01,4.03,0.58,3.45,2.20,3.58,0.88,2.70
435,9,Dhading,27.769034,84.854632,254.65,83.78,15.52,90.75,19.57,17.54,22.58,17.19,5.38,19.63,1.81,3.68,0.34,3.35,1.99,3.43,0.49,2.93


In [ ]:
train = train
target = target
test = test

# generate data
#new_train1, new_target1 = OriginalGenerator().generate_data_pipe(train, target, test, )
#new_train2, new_target2 = GANGenerator().generate_data_pipe(train, target, test, )

# example with all params defined
new_train3, new_target3 = GANGenerator(gen_x_times=5, cat_cols=['DISTRICT'], bot_filter_quantile=0.001,
                                       top_filter_quantile=0.999,
                                       is_post_process=True,
                                       adversaial_model_params={
                                           "metrics": "AUC", "max_depth": 2,
                                           "max_bin": 100, "n_estimators": 500,
                                           "learning_rate": 0.02, "random_state": 42,
                                       }, pregeneration_frac=2, only_generated_data=False,
                                       epochs=500).generate_data_pipe(train, target,
                                                                      test, deep_copy=True,
                                                                      only_adversarial=False,
                                                                      use_adversarial=True)

Fitting CTGAN transformers for each column:   0%|          | 0/23 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
new_train3

,MONTH,DISTRICT,latitude,longitude,PRECTOT,PS,QV2M,RH2M,T2M,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
0,4,Tanahun,28.157391,83.833867,452.813041,85.382144,18.750113,43.023807,26.747946,8.555607,10.040474,12.917138,6.408469,10.481437,2.450530,4.042796,0.475614,4.717698,2.761682,3.131584,1.463127,2.931324
1,7,Mustang,28.822601,85.118841,466.989381,95.701702,19.838977,25.278026,23.192299,6.561638,12.555718,11.755408,6.031107,13.039412,1.914308,3.772340,0.690532,3.441045,2.882032,3.204204,1.377739,3.106671
2,6,Saptari,28.494480,85.850762,490.067020,88.744474,17.578704,73.415883,21.778957,10.190243,12.991953,14.096072,8.321076,10.029551,2.210757,3.128973,0.608078,3.131865,2.062310,3.073474,0.801527,2.580421
3,11,Nuwakot,29.042936,87.738326,537.909317,85.262933,16.765206,42.144949,20.638368,12.834666,14.122057,9.159183,7.276719,-4.377686,2.103644,4.060743,0.703445,4.342155,1.932991,3.058684,1.222288,2.622531
4,6,Bara,28.289553,86.366823,203.589401,81.134918,2.083815,27.242007,23.149953,-15.540239,13.001341,17.670723,5.124314,-2.875158,3.170879,4.096025,0.291018,4.004003,3.956989,3.349095,0.708559,2.601341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,8,Bhaktapur,27.700000,85.500000,206.240000,77.740000,14.680000,90.790000,17.500000,15.420000,20.950000,14.650000,6.300000,18.240000,2.080000,4.490000,0.310000,4.180000,2.000000,3.790000,0.410000,3.380000
383,4,Saptari,26.600000,86.800000,2.750000,94.970000,10.160000,45.920000,26.190000,12.860000,32.370000,20.560000,11.810000,26.790000,2.680000,4.550000,0.890000,3.650000,3.510000,5.240000,1.420000,3.820000
384,9,Darchula,29.900000,80.600000,20.660000,73.880000,9.970000,71.530000,14.240000,8.150000,20.090000,9.700000,10.390000,14.990000,2.310000,5.170000,0.350000,4.820000,2.170000,4.300000,0.450000,3.850000
385,10,Jumla,29.300000,82.200000,0.140000,75.260000,5.110000,45.440000,11.100000,-0.980000,18.810000,5.500000,13.300000,10.260000,2.240000,4.840000,0.350000,4.500000,2.240000,4.260000,0.420000,3.840000


In [ ]:
new_target3

0      0
1      0
2      0
3      1
4      0
      ..
382    0
383    0
384    0
385    0
386    1
Name: output, Length: 387, dtype: int64

In [ ]:
final = new_train3.merge(new_target3, how='inner', on=new_target3.index)

In [ ]:
final

,key_0,MONTH,DISTRICT,latitude,longitude,PRECTOT,PS,QV2M,RH2M,T2M,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,output
0,0,4,Tanahun,28.157391,83.833867,452.813041,85.382144,18.750113,43.023807,26.747946,8.555607,10.040474,12.917138,6.408469,10.481437,2.450530,4.042796,0.475614,4.717698,2.761682,3.131584,1.463127,2.931324,0
1,1,7,Mustang,28.822601,85.118841,466.989381,95.701702,19.838977,25.278026,23.192299,6.561638,12.555718,11.755408,6.031107,13.039412,1.914308,3.772340,0.690532,3.441045,2.882032,3.204204,1.377739,3.106671,0
2,2,6,Saptari,28.494480,85.850762,490.067020,88.744474,17.578704,73.415883,21.778957,10.190243,12.991953,14.096072,8.321076,10.029551,2.210757,3.128973,0.608078,3.131865,2.062310,3.073474,0.801527,2.580421,0
3,3,11,Nuwakot,29.042936,87.738326,537.909317,85.262933,16.765206,42.144949,20.638368,12.834666,14.122057,9.159183,7.276719,-4.377686,2.103644,4.060743,0.703445,4.342155,1.932991,3.058684,1.222288,2.622531,1
4,4,6,Bara,28.289553,86.366823,203.589401,81.134918,2.083815,27.242007,23.149953,-15.540239,13.001341,17.670723,5.124314,-2.875158,3.170879,4.096025,0.291018,4.004003,3.956989,3.349095,0.708559,2.601341,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,382,8,Bhaktapur,27.700000,85.500000,206.240000,77.740000,14.680000,90.790000,17.500000,15.420000,20.950000,14.650000,6.300000,18.240000,2.080000,4.490000,0.310000,4.180000,2.000000,3.790000,0.410000,3.380000,0
383,383,4,Saptari,26.600000,86.800000,2.750000,94.970000,10.160000,45.920000,26.190000,12.860000,32.370000,20.560000,11.810000,26.790000,2.680000,4.550000,0.890000,3.650000,3.510000,5.240000,1.420000,3.820000,0
384,384,9,Darchula,29.900000,80.600000,20.660000,73.880000,9.970000,71.530000,14.240000,8.150000,20.090000,9.700000,10.390000,14.990000,2.310000,5.170000,0.350000,4.820000,2.170000,4.300000,0.450000,3.850000,0
385,385,10,Jumla,29.300000,82.200000,0.140000,75.260000,5.110000,45.440000,11.100000,-0.980000,18.810000,5.500000,13.300000,10.260000,2.240000,4.840000,0.350000,4.500000,2.240000,4.260000,0.420000,3.840000,0


In [ ]:
final['avg_windspeed'] = (final['WS10M'] + final['WS50M'])/2

In [ ]:
final

,key_0,MONTH,DISTRICT,latitude,longitude,PRECTOT,PS,QV2M,RH2M,T2M,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,output,avg_windspeed
0,0,4,Tanahun,28.157391,83.833867,452.813041,85.382144,18.750113,43.023807,26.747946,8.555607,10.040474,12.917138,6.408469,10.481437,2.450530,4.042796,0.475614,4.717698,2.761682,3.131584,1.463127,2.931324,0,2.606106
1,1,7,Mustang,28.822601,85.118841,466.989381,95.701702,19.838977,25.278026,23.192299,6.561638,12.555718,11.755408,6.031107,13.039412,1.914308,3.772340,0.690532,3.441045,2.882032,3.204204,1.377739,3.106671,0,2.398170
2,2,6,Saptari,28.494480,85.850762,490.067020,88.744474,17.578704,73.415883,21.778957,10.190243,12.991953,14.096072,8.321076,10.029551,2.210757,3.128973,0.608078,3.131865,2.062310,3.073474,0.801527,2.580421,0,2.136533
3,3,11,Nuwakot,29.042936,87.738326,537.909317,85.262933,16.765206,42.144949,20.638368,12.834666,14.122057,9.159183,7.276719,-4.377686,2.103644,4.060743,0.703445,4.342155,1.932991,3.058684,1.222288,2.622531,1,2.018318
4,4,6,Bara,28.289553,86.366823,203.589401,81.134918,2.083815,27.242007,23.149953,-15.540239,13.001341,17.670723,5.124314,-2.875158,3.170879,4.096025,0.291018,4.004003,3.956989,3.349095,0.708559,2.601341,0,3.563934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,382,8,Bhaktapur,27.700000,85.500000,206.240000,77.740000,14.680000,90.790000,17.500000,15.420000,20.950000,14.650000,6.300000,18.240000,2.080000,4.490000,0.310000,4.180000,2.000000,3.790000,0.410000,3.380000,0,2.040000
383,383,4,Saptari,26.600000,86.800000,2.750000,94.970000,10.160000,45.920000,26.190000,12.860000,32.370000,20.560000,11.810000,26.790000,2.680000,4.550000,0.890000,3.650000,3.510000,5.240000,1.420000,3.820000,0,3.095000
384,384,9,Darchula,29.900000,80.600000,20.660000,73.880000,9.970000,71.530000,14.240000,8.150000,20.090000,9.700000,10.390000,14.990000,2.310000,5.170000,0.350000,4.820000,2.170000,4.300000,0.450000,3.850000,0,2.240000
385,385,10,Jumla,29.300000,82.200000,0.140000,75.260000,5.110000,45.440000,11.100000,-0.980000,18.810000,5.500000,13.300000,10.260000,2.240000,4.840000,0.350000,4.500000,2.240000,4.260000,0.420000,3.840000,0,2.240000


In [ ]:
imp_features = ['MONTH', 'DISTRICT', 'latitude', 'longitude', 'PRECTOT', 'PS', 'RH2M', 'T2M', 'avg_windspeed', 'output']

In [ ]:
df['avg_windspeed'] = (df['WS10M'] + df['WS50M'])/2

In [ ]:
df = df[imp_features]

In [ ]:
final = final[imp_features]

In [ ]:
df

,MONTH,DISTRICT,latitude,longitude,PRECTOT,PS,RH2M,T2M,avg_windspeed,output
0,7,Syangja,27.914200,83.787600,413.84,85.41,88.30,22.02,2.235,1
1,6,Kaski,28.281201,83.864415,220.45,77.91,74.56,18.58,2.230,1
2,6,Salyan,28.400000,82.100000,214.13,88.50,59.08,26.01,3.500,0
3,8,Kathmandu,28.029700,85.093400,388.41,83.61,90.95,20.86,2.370,1
4,3,Baglung,28.308297,83.174393,66.43,79.29,47.96,10.37,2.390,1
...,...,...,...,...,...,...,...,...,...,...
510,7,Bardiya,28.450000,81.300000,18.58,93.90,78.74,28.20,2.820,0
511,7,Solukhumbu,27.500100,86.866900,119.03,56.27,75.37,6.87,2.650,1
512,4,Dhanusa,26.700000,85.900000,4.08,97.58,17.98,29.48,4.195,0
513,8,Gulmi,28.087700,83.097100,353.59,79.79,90.54,18.58,2.355,1


In [ ]:
data_final = df.append(final)

In [ ]:
data_final

,MONTH,DISTRICT,latitude,longitude,PRECTOT,PS,RH2M,T2M,avg_windspeed,output
0,7,Syangja,27.914200,83.787600,413.84,85.41,88.30,22.02,2.235,1
1,6,Kaski,28.281201,83.864415,220.45,77.91,74.56,18.58,2.230,1
2,6,Salyan,28.400000,82.100000,214.13,88.50,59.08,26.01,3.500,0
3,8,Kathmandu,28.029700,85.093400,388.41,83.61,90.95,20.86,2.370,1
4,3,Baglung,28.308297,83.174393,66.43,79.29,47.96,10.37,2.390,1
...,...,...,...,...,...,...,...,...,...,...
382,8,Bhaktapur,27.700000,85.500000,206.24,77.74,90.79,17.50,2.040,0
383,4,Saptari,26.600000,86.800000,2.75,94.97,45.92,26.19,3.095,0
384,9,Darchula,29.900000,80.600000,20.66,73.88,71.53,14.24,2.240,0
385,10,Jumla,29.300000,82.200000,0.14,75.26,45.44,11.10,2.240,0


In [ ]:
data_final.reset_index(drop=True)

,MONTH,DISTRICT,latitude,longitude,PRECTOT,PS,RH2M,T2M,avg_windspeed,output
0,7,Syangja,27.914200,83.787600,413.84,85.41,88.30,22.02,2.235,1
1,6,Kaski,28.281201,83.864415,220.45,77.91,74.56,18.58,2.230,1
2,6,Salyan,28.400000,82.100000,214.13,88.50,59.08,26.01,3.500,0
3,8,Kathmandu,28.029700,85.093400,388.41,83.61,90.95,20.86,2.370,1
4,3,Baglung,28.308297,83.174393,66.43,79.29,47.96,10.37,2.390,1
...,...,...,...,...,...,...,...,...,...,...
897,8,Bhaktapur,27.700000,85.500000,206.24,77.74,90.79,17.50,2.040,0
898,4,Saptari,26.600000,86.800000,2.75,94.97,45.92,26.19,3.095,0
899,9,Darchula,29.900000,80.600000,20.66,73.88,71.53,14.24,2.240,0
900,10,Jumla,29.300000,82.200000,0.14,75.26,45.44,11.10,2.240,0


In [ ]:
data_final.to_csv('final_data.csv')